In [1]:
import numpy as np
import scipy as sp
import qiskit
import math
import cmath

import matplotlib.pyplot as plt 

import numba as nb
from numba import jit

In [5]:
@jit(nopython=True)
def cnot1(nT, nA,nB):
    #Construct a CNOT with nT total qubits where the control is qubit A and the target is B

    z=np.array([[1,0],[0,0]])

    o=np.array([[0,0],[0,1]])

    X=np.array([[0,1],[1,0]])
    
    A=nA-1;
    m1=np.identity(2**A)

    B=nB-nA-1;
    m2=np.identity(2**B)

    C=nT-2-A-B
    m3=np.identity(2**C)

    matout1=np.kron(m1,z)
    matout2=np.kron(m2,np.identity(2))
    matout3=np.kron(matout1,matout2)
    matout4=np.kron(matout3,m3)

    matout11=np.kron(m1,o)
    matout21=np.kron(m2,X)
    matout31=np.kron(matout11,matout21)
    matout41=np.kron(matout31,m3)

    mattot=matout4+matout41

    return mattot

In [6]:
@jit(nopython=True)
def cnot2(nT, nA,nB):
    #Construct a CNOT with nT total qubits where the control is qubit B and the target is A

    z=np.array([[1,0],[0,0]])

    o=np.array([[0,0],[0,1]])

    X=np.array([[0,1],[1,0]])
    
    A=nA-1;
    m1=np.identity(2**A)

    B=nB-nA-1;
    m2=np.identity(2**B)

    C=nT-2-A-B
    m3=np.identity(2**C)

    matout1=np.kron(m1,np.identity(2))
    matout2=np.kron(m2,z)
    matout3=np.kron(matout1,matout2)
    matout4=np.kron(matout3,m3)

    matout11=np.kron(m1,X)
    matout21=np.kron(m2,o)
    matout31=np.kron(matout11,matout21)
    matout41=np.kron(matout31,m3)

    mattot=matout4+matout41

    return mattot

In [7]:
@jit(nopython=True)
def u3(th,phi,lam):
    #Construct an arbitrary single qubit unitary

    u3=np.array([[math.cos(th/2),-cmath.exp(1j*lam)*math.sin(th/2)],[cmath.exp(1j*phi)*math.sin(th/2),cmath.exp(1j*(lam+phi))*math.cos(th/2)]])
    return u3

In [8]:
from numba import njit, complex128

In [9]:
#two qubit parameterised circuit - we may need to try different structures - this 2 qubit unitary acts on the final two qubits at the output
@jit(nopython=True)
def par2(layers,x):
    
    utot=np.identity(4)+0*1j*np.identity(4)
    for ii in range(layers):
        # xv1=x[0][0+ii*6:6*(ii+1)]
        xv1=x[0+ii*6:6*(ii+1)]
        #print(xv1)
        # u1=u3(xv1[0],xv1[1],xv1[2])
        # u2=u3(xv1[3],xv1[4],xv1[5])
        u1=u3(xv1[0],0,0)
        u2=u3(xv1[3],0,0)
        uind=np.kron(u1,u2)
        unew=uind@utot
        #utot=np.dot(uind, utot)
        # unew=dot_nb(uind, utot)
        utot=unew
        if (ii%2)==0:
            cnot=cnot2(2, 1,2)+0*1j*np.identity(4)
        else:
            cnot=cnot1(2, 1,2)+0*1j*np.identity(4)
        #unew=dot_nb(cnot, utot)
        
        #utot=np.dot(cnot, utot)
        utot=cnot@utot
        utot=unew
        
    return utot

In [13]:
#four qubit parameterised circuit - we may need to try different structures - this acts on the two input qubits
#and two arms of the entangled state
@jit(nopython=True)
def par4(layers,x):
    
    utot=np.identity(16)+0*1j*np.identity(16)
    for ii in range(layers):
        
        xv1=x[0+ii*12:12*(ii+1)]
        #print(xv1)
        u1=u3(xv1[0],xv1[1],xv1[2])
        u2=u3(xv1[3],xv1[4],xv1[5])
        u33=u3(xv1[6],xv1[7],xv1[8])
        u4=u3(xv1[9],xv1[10],xv1[11])
        
        uind1=np.kron(u1,u2)
        uind2=np.kron(u33,u4)
        uind=np.kron(uind1,uind2)
        utot=uind@utot
        if (ii%4)==0:
            cnot11=cnot1(4, 1,2)+0*1j*np.identity(16)
            cnot22=cnot2(4, 3,4)+0*1j*np.identity(16)
        elif (ii%4)==1:
            cnot11=cnot1(4, 2,3)+0*1j*np.identity(16)
            cnot22=cnot2(4, 1,4)+0*1j*np.identity(16)
        elif (ii%4)==2:
            cnot11=cnot2(4, 1,3)+0*1j*np.identity(16)
            cnot22=cnot2(4, 2,4)+0*1j*np.identity(16)
        elif (ii%4)==3:
            cnot11=cnot2(4, 1,2)+0*1j*np.identity(16)
            cnot22=cnot1(4, 3,4)+0*1j*np.identity(16)
        utot=cnot22@cnot11@utot
        
    return utot

In [14]:
# make bell state
@jit(nopython=True)
def bell(th):
    psibell=math.cos(th)*np.array([1+0*1j,0+0*1j,0+0*1j,0+0*1j])+math.sin(th)*np.array([0+0*1j,0+0*1j,0+0*1j,1+0*1j])
    return psibell

#input states
@jit(nopython=True)
def inputf(x):
    if x==1:
        insd=np.array([1+0*1j,0+0*1j])
    elif x==2:
        insd=np.array([1+0*1j,1+0*1j])/(2**(0.5))
    elif x==3:
        insd=np.array([1+0*1j,0+1j])/(2**(0.5))
    elif x==4:
        insd=np.array([0+0*1j,1+0*1j])
    elif x==5:
        insd=np.array([1+0*1j,-1+0*1j])/(2**(0.5))
    elif x==6:
        insd=np.array([1+0*1j,0-1j])/(2**(0.5))
    return insd

In [15]:
#projector operators for the single-copy teleporter. Depending on the measurement outcome we project onto a different 
#state and then implement a different single qubit operation at the output
@jit(nopython=True)
def projector(inputstate):
    p00=np.array([1,0,0,0])+0*1j*np.array([1,0,0,0])
    p01=np.array([0,1,0,0])+0*1j*np.array([1,0,0,0])
    p10=np.array([0,0,1,0])+0*1j*np.array([1,0,0,0])
    p11=np.array([0,0,0,1])+0*1j*np.array([1,0,0,0])

    proj00=np.kron(p00,np.identity(2)*(1+0*1j))
    proj01=np.kron(p01,np.identity(2)*(1+0*1j))
    proj10=np.kron(p10,np.identity(2)*(1+0*1j))
    proj11=np.kron(p11,np.identity(2)*(1+0*1j))

    state00=proj00@inputstate
    prob00=state00@np.conj(np.transpose(state00))
    
    state01=proj01@inputstate
    prob01=state01@np.conj(np.transpose(state01))
    
    state10=proj10@inputstate
    prob10=state10@np.conj(np.transpose(state10))
    
    state11=proj11@inputstate
    prob11=state11@np.conj(np.transpose(state11))
    
    return state00, state01, state10, state11, prob00, prob01, prob10, prob11

In [16]:
#projectors for 2 qubits
@jit(nopython=True)
def projector4(ind):
    #p=np.full([1,16], 0+0*1j)
    p=np.array([[0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j,0+0*1j]])
    p[0][ind]=1
    return p
    
@jit(nopython=True)
def projectedstates4(inputstate):
    statevec=[]
    probvec=[]
    for kk in range(16):
        projint=np.kron(projector4(kk),np.identity(4)*(1+0*1j))

        stateint=projint@inputstate
        probint=stateint@np.conj(np.transpose(stateint))

        statevec.append(stateint)
        probvec.append(probint)

    return statevec, probvec

In [17]:
# for the 2-copy teleporter we will need to swap qubits 4 and 5 
swap=np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])*(1+0*1j)
swap45=np.kron(np.identity(8),np.kron(swap,np.identity(2)))*(1+0*1j)

In [18]:
@jit(nopython=True)
def totalstate4(theta,statein1,statein2):
    #total input state for the 2-copy teleporter
    bells=bell(theta)
    stin1=inputf(statein1)
    stin2=inputf(statein2)
    totalinput1=np.kron(stin1,stin2)
    totalinput2=np.kron(bells,bells)
    
    totalinput=np.kron(totalinput1,totalinput2)

    finalinput=swap45@totalinput
    return finalinput,stin1,stin2

In [20]:
#single-copy teleporter fidelity averaged over the 6 axes states of the Bloch vector
@jit(nopython=True)
def fid(x):
    fid=0
    for kk in range(6):
        statein=kk+1
        bells=bell(theta)
        stin=inputf(statein)
        totalinput=np.kron(stin,bells)
        #two qubit operation acting on one arm of the bell state and the input state
        utest=par2(layers,x)
        [s00,s01,s10,s11,p00,p01,p10,p11]=projector(np.kron(utest,np.identity(2))@np.transpose(totalinput))

        #now a single qubit unitary for each measurement outcome
        uend1=u3(x[6*layers],x[6*layers+1],x[6*layers+2])
        uend2=u3(x[6*layers+3],x[6*layers+4],x[6*layers+5])
        uend3=u3(x[6*layers+6],x[6*layers+7],x[6*layers+8])
        uend4=u3(x[6*layers+9],x[6*layers+10],x[6*layers+11])

        fid=fid+np.abs((stin@np.conj(np.transpose(uend1@s00)))**2)+np.abs((stin@np.conj(np.transpose(uend2@s01)))**2)+np.abs((stin@np.conj(np.transpose(uend3@s10)))**2)+np.abs((stin@np.conj(np.transpose(uend4@s11)))**2)
    return -fid/6

In [21]:
#2-copy teleporter fidelity averaged over the 6^2 tensor products of the 6 axes states of the Bloch vector
@jit(nopython=True)
def fid4(x):
    fid=0
    for hh in range(6):
        for gg in range(6):
            
            [inputstate4circ,stin1,stin2]=totalstate4(theta,hh+1,gg+1)
            targetstate=np.kron(stin1,stin2)
            layers=4
            
            #x=np.random.rand(1,12*layers+16*6)
            
            utest=par4(layers,x)
            
            [states,probs]=projectedstates4(np.kron(utest,np.identity(4))@np.transpose(inputstate4circ))
        
            for ii in range(16):
                outputstateint=states[ii]
                uend11=u3(x[12*layers+(ii)*6],x[12*layers+(ii+1)*1],x[12*layers+(ii+1)*2])
                uend12=u3(x[12*layers+(ii+1)*3],x[12*layers+(ii+1)*4],x[12*layers+(ii+1)*5])
                outputstatefin=np.kron(uend11,uend12)@outputstateint
                fid=fid+np.abs((targetstate@np.conj(np.transpose(outputstatefin)))**2)

    return -fid/36

In [22]:
#test
theta=math.pi/4-0.1;

layers=4;
x=np.random.rand(1,12*layers+(2**4)*6)
fid4(x[0])

-0.23888068634415818

In [ ]:
#theta controls the degree of entanglement in the bell state. We optimise the PQC to maximise fidelity at each theta value
thvec=np.linspace(math.pi/10,math.pi/4,5)
#layers

l3=4;
l4=4;

fidvec2=[]
fidvec14q=[]
fidvec24q=[]
for kk in range(len(thvec)):
    print(kk)
    theta=thvec[kk]
    
    layers=l3
    x0=np.random.rand(1,6*layers+12)
    F2=sp.optimize.minimize(fid,x0)
    fidvec2.append(-F2.fun)

    layers=l4
    x0=np.random.rand(1,12*layers+16*6)
    F4=sp.optimize.minimize(fid4,x0,method='Powell')
    fidvec24q.append(-F4.fun)

    

    print(-F2.fun,-F4.fun)#,-F6.fun)

# plt.plot(thvec,fidvec1,'r-o')

plt.plot(thvec,fidvec2,'b-*')

# plt.plot(thvec,fidvec14q,'k-o')

plt.plot(thvec,fidvec24q,'g-*')
